In [87]:
import pandas as pd
import re
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV


In [88]:
import tweepy,csv,time

ACCESS_TOKEN = '938493239853813762-tlImlRTGU6RraqmBQQQQ3K5klFmHwvG'
ACCESS_SECRET = 'Ga79wDf0YK9YtYc21XEfBH10L0wAttnDL5wQXVT7oNj1J'
CONSUMER_KEY = 'zHK9haWtcpGWHUq12fEMn3Fyw'
CONSUMER_SECRET = 'jIZj1D7jPfXPGSIQTTEGMPgXhIR7lEqyBXRUnudcgpvPDc2wzm'

auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACCESS_SECRET)

api = tweepy.API(auth)

In [106]:
# Tool to delete unicode codes
def unicodetoascii(text):
    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').#
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")")
                 )
    return TEXT

#Reading CSV file
dataset=pd.read_csv('tweets3.csv', sep=',',quotechar='"',header=None,)
#Replacing Unicode characters
dataset[2]=dataset[2].apply(unicodetoascii)
#Deleting Urls
delete_urls = lambda x:re.sub(r'http\S+', '', x)
dataset[2] = dataset[2].apply(delete_urls)
#Deleting "
delete_quotes = lambda x:re.sub(r'"', '', x)
dataset[2] = dataset[2].apply(delete_quotes)
#Deleting b
delete_b = lambda x:re.sub(r'b\'', '', x)
dataset[2] = dataset[2].apply(delete_b)

#Text cleaning
def clean (X):
    X=delete_urls(X)
    X=delete_quotes(X)
    X=delete_b(X)
    return X
#Datasets
X=dataset[2].values
Y=dataset[3].values


In [107]:
#Data split
x_train, x_test ,y_train,y_test = train_test_split(X,Y, test_size=0.3)

In [108]:
#Machine Learning NB
text_clf_NB = Pipeline([('vect', CountVectorizer(stop_words='english')),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])
text_clf_NB = text_clf_NB.fit(x_train, y_train)

# Performance
y_predicted = text_clf_NB.predict(x_test)
np.mean(y_predicted == y_test)
# 78% accuracy

0.7825393661550728

In [109]:
#Machine Learning SVM
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english')),
                   ('tfidf', TfidfTransformer()),
                   ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42)), ])

_ = text_clf_svm.fit(x_train,y_train)
predicted_svm = text_clf_svm.predict(x_test)
np.mean(predicted_svm == y_test)
# 71% accuracy

/home/loko/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.72055012955949771

In [110]:
#Grid search NB
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(text_clf_NB, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(x_train,y_train)
print(gs_clf.best_score_)
gs_clf.best_params_
#78% accuracy

0.786126180001


{'clf__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}

In [111]:
#Grid search SVM
from sklearn.model_selection import GridSearchCV
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(x_train,y_train)
print(gs_clf_svm.best_score_)
gs_clf_svm.best_params_
#71% accuracy

0.719405407714


{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}

In [112]:
# Input : text + pipeline(fitted) , output : prediction + accuracy (works only with NB classifier)
def tweet_prediction (txt,func):
    iter = np.array([txt])
    if (func.predict(iter)[0] == 1):
        print("This tweet is recognized as democrat with a "+str(round(func.predict_proba(iter)[0][1]*100,2))+" % certainty")
        return 1
    else:
        print("This tweet is recognized as republican with a "+str(round(func.predict_proba(iter)[0][0]*100,2))+" % certainty")
        return -1
    
tweet_prediction("Trump",text_clf_NB)   

This tweet is recognized as democrat with a 78.28 % certainty


1

In [131]:
# Input : tweeter account + pipeline (fitted), output : prediction 
api = tweepy.API(auth)
def account_prediction (acc,func):
    iter=np.array([])
    tweets = api.user_timeline(acc,count=10000)
    time.sleep(0.1)
    for tweet in tweets:
        if ('RT @' not in tweet.text):
            iter=np.append([clean(tweet.text)],iter)
            
    result = func.predict(iter)
    ratio = np.mean(result)
    #print (ratio)
    if ratio <0 :
        print("The account @"+acc+" is recognized as a republican account.")
    else:
        print("The account @"+acc+" is recognized as a democrat account.")
        
account_prediction("RepMiaLove",text_clf_NB)

The account @RepMiaLove is recognized as a republican account.
